<a href="https://colab.research.google.com/github/Dinesh9928/Data_science_protfolio/blob/main/StockMarket_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

Loading main data

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
data = yf.download("GOOGL" , start = "2018-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [ ]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,1053.020020,1075.979980,1053.020020,1073.209961,1073.209961,1588300
2018-01-03,1073.930054,1096.099976,1073.430054,1091.520020,1091.520020,1565900
2018-01-04,1097.089966,1104.079956,1094.260010,1095.760010,1095.760010,1302600


In [ ]:
# Sort the data points based on indexes just for confirmation 
data.sort_index(inplace = True)

In [ ]:
# Remove any duplicate index 
data = data.loc[~data.index.duplicated(keep='first')]

In [ ]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-01-13,1727.319946,1756.390015,1727.319946,1747.250000,1747.250000,1171600
2021-01-14,1748.869995,1767.550049,1726.069946,1730.920044,1730.920044,1460600
2021-01-15,1729.020020,1747.619995,1711.709961,1727.619995,1727.619995,1570200


In [ ]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,1053.020020,1075.979980,1053.020020,1073.209961,1073.209961,1588300
2018-01-03,1073.930054,1096.099976,1073.430054,1091.520020,1091.520020,1565900
2018-01-04,1097.089966,1104.079956,1094.260010,1095.760010,1095.760010,1302600
2018-01-05,1103.449951,1113.579956,1101.800049,1110.290039,1110.290039,1512500
2018-01-08,1111.000000,1119.160034,1110.000000,1114.209961,1114.209961,1232200


In [ ]:
# Check for missing values 
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# Get the statistics of the data
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,766.000000,766.000000,766.000000,766.000000,766.000000,7.660000e+02
mean,1270.177141,1283.951957,1256.716451,1270.842833,1270.842833,1.849574e+06
std,200.598166,202.740271,199.361301,201.046614,201.046614,8.646551e+05
min,984.320007,1012.119995,977.659973,984.669983,984.669983,4.656000e+05
25%,1118.109985,1129.475037,1106.774963,1116.962463,1116.962463,1.301775e+06
50%,1203.049988,1213.449951,1193.285034,1206.254944,1206.254944,1.631750e+06
75%,1410.297485,1427.847534,1393.950012,1412.037537,1412.037537,2.114500e+06
max,1820.540039,1843.829956,1817.000000,1824.969971,1824.969971,6.658900e+06


Understanding Trends with in the Data

In [ ]:
import plotly.graph_objects as go

In [ ]:
# Check the trend in Closing Values 
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()

Data Preparation

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
import pickle 
from tqdm.notebook import tnrange

In [ ]:
# Filter only required data 
data = data[['Close' , 'Volume']]
data.head(3)

,Close,Volume
Date,,
2018-01-02,1073.209961,1588300
2018-01-03,1091.520020,1565900
2018-01-04,1095.760010,1302600


Scrapping extra information

In [ ]:
import requests 

response = requests.get('https://www.alphavantage.co/query?function=RSI&symbol=GOOGL&interval=daily&time_period=5&series_type=close&apikey=43T9T17VCV2ME4SM') 
response = response.json()

In [ ]:
response.keys()

dict_keys(['Meta Data', 'Technical Analysis: RSI'])

In [ ]:
rsi_data = pd.DataFrame.from_dict(response['Technical Analysis: RSI'] , orient='index')

In [ ]:
rsi_data.head()

,RSI
2004-08-26,72.7683
2004-08-27,64.2686
2004-08-30,47.8388
2004-08-31,49.2490
2004-09-01,41.0745


In [ ]:
rsi_data = rsi_data[rsi_data.index >= '2018-01-01']

In [ ]:
rsi_data['RSI'] = rsi_data['RSI'].astype(np.float64)

In [ ]:
rsi_data.head()

,RSI
2018-01-02,65.4310
2018-01-03,78.3686
2018-01-04,80.4830
2018-01-05,86.2429
2018-01-08,87.4882


In [ ]:
data = data.merge(rsi_data, left_index=True, right_index=True, how='inner')

In [ ]:
data.head()

,Close,Volume,RSI
Date,,,
2018-01-02,1073.209961,1588300,65.4310
2018-01-03,1091.520020,1565900,78.3686
2018-01-04,1095.760010,1302600,80.4830
2018-01-05,1110.290039,1512500,86.2429
2018-01-08,1114.209961,1232200,87.4882


In [ ]:
# Confirm the Testing Set length 
test_length = data[(data.index >= '2020-09-01')].shape[0]

In [ ]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [ ]:
X , Y = CreateFeatures_and_Targets(data , 32)

In [ ]:
# Check the shapes
X.shape , Y.shape

((734, 32, 3), (734,))

In [ ]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [ ]:
# Check Training Dataset Shape 
Xtrain.shape , Ytrain.shape

((639, 32, 3), (639,))

In [ ]:
# Check Testing Dataset Shape
Xtest.shape , Ytest.shape

((95, 32, 3), (95,))

In [ ]:
# Create a Scaler to Scale Vectors with Multiple Dimensions 
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:,:,i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [ ]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [ ]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [ ]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [ ]:
# Save your objects for future purposes 
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

Model Building

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=5, min_lr=0.00001,verbose = 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional , BatchNormalization

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 3))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [ ]:
history = model.fit(Xtrain, Ytrain,
            epochs=10,
            batch_size = 1,
            verbose=1,
            shuffle=False ,
            validation_data=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

20/20 [==============================] - 18s 500ms/step - loss: 0.0443 - val_loss: 0.3863


In [ ]:
# Load the best weights
model.load_weights("best_weights.h5")

Visualize prediction on Test Set

In [ ]:
Predictions = model.predict(Xtest)

In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [ ]:
Predictions.shape

(95, 1)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:
# Check the Predictions vs Actual
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index[-test_length:] , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index[-test_length:] , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

Visualize Prediction on whole data

In [ ]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [ ]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [ ]:
Predictions = model.predict(Total_features)

In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

In [ ]:
# Save and Load the whole model
model.save("Model.h5")
loaded_model = tf.keras.models.load_model("Model.h5")

Realtime Prediction

In [ ]:
def PredictStockPrice(Model , DataFrame , PreviousDate , feature_length = 32):
    idx_location = DataFrame.index.get_loc(PreviousDate)
    Features = DataFrame.iloc[idx_location - feature_length : idx_location,:].values
    Features = np.expand_dims(Features , axis = 0)
    Features = Feature_Scaler.transform(Features)
    Prediction = Model.predict(Features)
    Prediction = Target_Scaler.inverse_transform(Prediction)
    return Prediction[0][0]

In [ ]:
PredictStockPrice(loaded_model , data , '2021-01-14')

1264.928